In [1]:
import ray
from ray.rllib.algorithms.sac import SACConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym

In [2]:
import ray
from ray.rllib.algorithms.es import ESConfig
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray import air, tune
from ray.air import session
from ray.air.integrations.wandb import setup_wandb
from ray.air.integrations.wandb import WandbLoggerCallback

import gym

In [3]:
# Test that mujoco for gym is available
gym.make("HalfCheetah-v3")

<TimeLimit<HalfCheetahEnv<HalfCheetah-v3>>>

In [4]:
config = (
    ESConfig()
    .environment(
        env="HalfCheetah-v3"
    )
    .rollouts(
        num_rollout_workers=3
    )
    .resources(num_gpus=0)
    # .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .framework(framework="torch")
)

In [5]:
config = (
    ESConfig()
    .environment(
        env="HalfCheetah-v3"
    )
    .rollouts(
        num_rollout_workers=3,
        num_envs_per_worker=8,
        recreate_failed_workers=True,
        num_consecutive_worker_failures_tolerance=10,
        restart_failed_sub_environments=True,
    )
    .resources(num_gpus=0)
    # .evaluation(evaluation_interval=100) # For 1000 timesteps iter; 100 evals
    .framework(framework="torch")
)
config.to_dict()

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 0,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': 'HalfCheetah-v3',
 'env_config': {},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 'num_envs_per_worker': 8,
 'sample_collector': ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector,
 'sampl

In [6]:
wandb_init = dict(
    save_code=True,
    config={
        "env": "HalfCheetah-v3",
        "num_rollout_workers": 3,
        "num_envs_per_worker"=8,
        "recreate_failed_workers"=True,
        "num_consecutive_worker_failures_tolerance"=10,
        "restart_failed_sub_environments"=True,
        "num_gpus": 0
        "framework": "torch"
    },
    tags=["local"],
    notes="Test to inspect scaling on Vast.ai",
    name="HalfCheetah_local"
    # job_type=D
    # monitor_gym=
)

In [7]:
wandb_init = dict(
    save_code=True,
    config={
        "env": "HalfCheetah-v3",
        "num_rollout_workers": 3,
        "num_envs_per_worker": 8,
        "recreate_failed_workers": True,
        "num_consecutive_worker_failures_tolerance": 10,
        "restart_failed_sub_environments": True,
        "num_gpus": 0,
        "framework": "torch"
    },
    tags=["local"],
    notes="Test to inspect scaling on Vast.ai",
    name="HalfCheetah_local"
    # job_type=D
    # monitor_gym=
)

In [8]:
tuner = tune.Tuner(
    "ES",
    run_config=air.RunConfig(
        name="HalfCheetah_local",
        stop={"episode_reward_mean": 150},
        checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
        callbacks=[
                WandbLoggerCallback(project="HalfCheetah", 
                                    api_key="392c8a47eb0658eb5c71190757a69110e2140f4a",
                                    save_checkpoints=True, 
                                    **wandb_init)
            ],
        local_dir="./results"
        ),
    param_space=config.to_dict()
)

results = tuner.fit()